In [ ]:
import pandas as pd
from db_queries import get_location_metadata

import warnings
warnings.simplefilter('ignore')


## get pct change in predicted deaths

In [ ]:
loc_df = get_location_metadata(location_set_id=111, location_set_version_id=664)
us_locs = loc_df.loc[loc_df['path_to_top_parent'].str.startswith('102,'), 'location_id'].to_list()

df = pd.read_csv('/ihme/covid-19/deaths/dev/2020_05_16_mrtool/model_results.csv')
df = df.loc[df['location_id'].isin(us_locs)]
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].apply(lambda x: x - pd.Timedelta(days=8))
df['mean_deaths'] = np.nan
df['mean_deaths'][1:] = np.diff(df[[f'draw_{d}' for d in range(1000)]], axis=0).mean(axis=1)
df = df.loc[df['date'] != df.groupby('location_id')['date'].transform(min)]

start_idx = df['date'] == pd.to_datetime('2020-05-10')
end_idx = df['date'] == df.groupby('location_id')['date'].transform(max)

df = df.loc[start_idx | end_idx, ['location_id', 'date', 'mean_deaths']]
df['last_day_of_data'] = df.groupby('location_id')['date'].transform(max)
df['date'] = df['date'].apply(lambda x: 'start' if x == pd.to_datetime('2020-05-10') else 'end')

df = pd.pivot_table(df, index=['location_id', 'last_day_of_data'], values=['mean_deaths'], columns='date').reset_index()
df.columns = df.columns.get_level_values(0)[:2].to_list() + df.columns.get_level_values(1)[2:].to_list()

df['pct_change_0510'] = ((df['end'] - df['start']) / df['start']) * 100

df = loc_df[['location_id', 'location_name']].merge(df[['location_id', 'last_day_of_data', 'pct_change_0510']])

df.to_csv('/ihme/covid-19/deaths/dev/2020_05_16_mrtool/pct_change_0510.csv', index=False)


In [ ]:
df